In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

!pip install openpyxl
!pip install pulp

import pulp

from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
OrderList = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='OrderList')
FreightRates = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='FreightRates')
WhCosts = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='WhCosts')
WhCapacities = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='WhCapacities')
ProductsPerPlant = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='ProductsPerPlant')
VmiCustomers = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='VmiCustomers')
PlantPorts = pd.read_excel('/content/drive/MyDrive/ACO/Supply chain logisitcs problem.xlsx', sheet_name='PlantPorts')

In [ ]:
# Data cleaning for OrderList

# OrderList format conversion
OrderList['TPT'] = OrderList['TPT'].astype(int)
OrderList['Ship ahead day count'] = OrderList['Ship ahead day count'].astype(int)
OrderList['Ship Late Day count'] = OrderList['Ship Late Day count'].astype(int)
OrderList['Unit quantity'] = OrderList['Unit quantity'].astype(int)
OrderList['Weight'] = OrderList['Weight'].astype(float)
# drop nulls form OrderList
OrderList.dropna(inplace=True)
# drop duplicates from OrderList
OrderList.drop_duplicates(inplace=True)

# Data cleaning for FreightRates

# FreightRates format conversion
FreightRates['minm_wgh_qty'] = FreightRates['minm_wgh_qty'].astype(int)
FreightRates['max_wgh_qty'] = FreightRates['max_wgh_qty'].astype(float)
FreightRates['minimum cost'] = FreightRates['minimum cost'].astype(float)
FreightRates['rate'] = FreightRates['rate'].astype(float)
FreightRates['tpt_day_cnt'] = FreightRates['tpt_day_cnt'].astype(int)
# drop nulls form FreightRates
FreightRates.dropna(inplace=True)
# drop duplicates from FreightRates
FreightRates.drop_duplicates(inplace=True)

# Data cleaning for WhCosts

# WhCosts format conversion
WhCosts['Cost/unit'] = WhCosts['Cost/unit'].astype(float)
# drop nulls form WhCosts
WhCosts.dropna(inplace=True)
# drop duplicates from WhCosts
WhCosts.drop_duplicates(inplace=True)

# Data cleaning for WhCapacities

# WhCapacities format conversion
WhCapacities['Daily Capacity '] = WhCapacities['Daily Capacity '].astype(int)
# drop nulls form WhCapacities
WhCapacities.dropna(inplace=True)
# drop duplicates from WhCapacities
WhCapacities.drop_duplicates(inplace=True)

# Data cleaning for ProductsPerPlant

# drop nulls form ProductsPerPlant
ProductsPerPlant.dropna(inplace=True)
# drop duplicates from ProductsPerPlant
ProductsPerPlant.drop_duplicates(inplace=True)

# Data cleaning for VmiCustomers

# drop nulls form VmiCustomers
VmiCustomers.dropna(inplace=True)
# drop duplicates from VmiCustomers
VmiCustomers.drop_duplicates(inplace=True)

# Data cleaning for

# drop nulls form PlantPorts
PlantPorts.dropna(inplace=True)
# drop duplicates from PlantPorts
PlantPorts.drop_duplicates(inplace=True)

In [ ]:
print(OrderList.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9215 entries, 0 to 9214
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Order ID              9215 non-null   float64       
 1   Order Date            9215 non-null   datetime64[ns]
 2   Origin Port           9215 non-null   object        
 3   Carrier               9215 non-null   object        
 4   TPT                   9215 non-null   int64         
 5   Service Level         9215 non-null   object        
 6   Ship ahead day count  9215 non-null   int64         
 7   Ship Late Day count   9215 non-null   int64         
 8   Customer              9215 non-null   object        
 9   Product ID            9215 non-null   int64         
 10  Plant Code            9215 non-null   object        
 11  Destination Port      9215 non-null   object        
 12  Unit quantity         9215 non-null   int64         
 13  Weight            

In [ ]:
print(FreightRates.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537 entries, 0 to 1539
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Carrier       1537 non-null   object 
 1   orig_port_cd  1537 non-null   object 
 2   dest_port_cd  1537 non-null   object 
 3   minm_wgh_qty  1537 non-null   int64  
 4   max_wgh_qty   1537 non-null   float64
 5   svc_cd        1537 non-null   object 
 6   minimum cost  1537 non-null   float64
 7   rate          1537 non-null   float64
 8   mode_dsc      1537 non-null   object 
 9   tpt_day_cnt   1537 non-null   int64  
 10  Carrier type  1537 non-null   object 
dtypes: float64(3), int64(2), object(6)
memory usage: 144.1+ KB
None


In [ ]:
print(WhCosts.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   WH         19 non-null     object 
 1   Cost/unit  19 non-null     float64
dtypes: float64(1), object(1)
memory usage: 456.0+ bytes
None


In [ ]:
print(WhCapacities.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Plant ID         19 non-null     object
 1   Daily Capacity   19 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 456.0+ bytes
None


In [ ]:
print(ProductsPerPlant.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2036 entries, 0 to 2035
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Plant Code  2036 non-null   object
 1   Product ID  2036 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.7+ KB
None


In [ ]:
print(VmiCustomers.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 13
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Plant Code  14 non-null     object
 1   Customers   14 non-null     object
dtypes: object(2)
memory usage: 336.0+ bytes
None


In [ ]:
print(PlantPorts.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 0 to 21
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Plant Code  22 non-null     object
 1   Port        22 non-null     object
dtypes: object(2)
memory usage: 528.0+ bytes
None


# ***Fitness function***

most cost effective supply chain


    total_cost = Min (WC(ki)+TC(kpj))


WC(ki)


    warehouse_cost_for_order_k_at_warehouse_i = number_of_units_in_order_qk * warehouse_storage_rate_Pi
(from WhCosts table)

TC(kpj)

transportation cost TC(kpj) for a given order k and
chosen line between origin port p and destination port j

    If s(k) = CRF

      then TC(kpj) = 0

    else if m = GROUND

      then TC(kpj)  = R(pjcstm) / sigma[k=1 to l] W(kpjcstm) * W(kpjcstm)

    else

      then TC(kpj) = R(pjcstm) * W(kpjcstm)

      if TC(kpj) < M(pjcstm)

        then TC(kpj) = M(pjcstm)



# S(k)

is the service level for order k, p – origin port, j– destination port, c – courier, s – service level, t – delivery time, m – transportation mode

# M(pjcstm)

is the minimum charge for given
line pjcstm

# W(kpjcstm)

is the weight in kilograms for order k. 

# R(pjcstm)

is thefreight rate (dollars per kilogram) for given weight gap based on the total weight for the line pjcstm (FreightRates table)
___
The algorithm first checks what kind of service level the order requires, if the service level sk is equal to CRF (Customer Referred
Freight) – transportation cost is 0. Furthermore, if order transportation
mode m is equal to GROUND (order transported via truck), order
transportation cost is proportional to the weight consumed by the order
(wkpjcstm) in respect of the total weight for given line pjcstm and the rate
charged by a courier for full track Rpjcstm. In all other cases, the transportation cost is calculated based on order weight wkpjcstm and the
freight rate Rpjcstm. The freight rate is determined based on total weight
on any given line pjcstm and the corresponding weight band in the
freight rate table. Furthermore, a minimum charge Mpjcstm is applied in
cases where the air transportation cost is less than the minimum charge.

# ***Restrictions***

    SIGMA [ k = 1 to l ] o(ki) <= C(i)

where oki = 1 if order k was shipped from warehouse i and 0 otherwise.
Ci is the order limit per day for warehouse i (WhCapacities table)

    SIGMA [ k = 1 to l ] W(kpjcstm) <= MAX F(pjcstm)

where wkpjcst is the weight in kilograms for order k shipped from
warehouse port p to customer port j via courier c using service level s,
delivery time t and transportation mode m. Fpjcstm is the upper weight
gap limit for line pjcstm (FreightRates table).

    K(z) is in I(z)

where product z for order k belongs to supported products at warehouse i (ProductsPerPlant table). Warehouses can only support given
customer in the VmiCustomers table, while all other warehouses that are
not in the table can supply any customer. Moreover, the warehouse can
only ship orders via supported origin port, defined in PlantPorts table